# Training SimpleNN on CIFAR-10
In this project, you will use the SimpleNN model to perform image classification on CIFAR-10. CIFAR-10 orginally contains 60K images from 10 categories. We split it into 45K/5K/10K images to serve as train/valiation/test set. We only release the ground-truth labels of training/validation dataset to you.

## Step 0: Set up the SimpleNN model
As you have practiced to implement simple neural networks in Homework 1, we just prepare the implementation for you.

In [1]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
print(torch.__version__)
# Check CUDA version supported by PyTorch
print("CUDA version supported by PyTorch:", torch.version.cuda)

2.4.1+cu124
CUDA version supported by PyTorch: 12.4


In [3]:
def check_cuda():
    if torch.cuda.is_available():
        print(
            "CUDA is available. You have", torch.cuda.device_count(), "CUDA device(s)."
        )
        for i in range(torch.cuda.device_count()):
            print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("CUDA is not available.")


check_cuda()

CUDA is available. You have 1 CUDA device(s).
Device 0: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
# define the SimpleNN mode;
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.silu = nn.SiLU()

    def forward(self, x):
        out = self.silu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out, 2)
        out = self.silu(self.bn2(self.conv2(out)))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.silu(self.fc1(out))
        out = self.silu(self.fc2(out))
        out = self.fc3(out)
        return out

### Question (a)
Here is a sanity check to verify the implementation of SimpleNN. 
You need to:
1. Write down your code.
2. **In the PDF report**, give a brief description on how the code helps you know that SimpleNN is implemented correctly.

In [5]:
#############################################
# your code here
# sanity check for the correctness of SimpleNN
# each image for CIFAR is a 32x32 RGB image so the input size is 3x32x32
# the output size should be 10
model = SimpleNN()
x = torch.randn(1, 3, 32, 32)
output = model(x)
print(output.size())
#############################################

torch.Size([1, 10])


## Step 1: Set up preprocessing functions
Preprocessing is very important as discussed in the lecture.
You will need to write preprocessing functions with the help of *torchvision.transforms* in this step.
You can find helpful tutorial/API at [here](https://pytorch.org/vision/stable/transforms.html).

### Question (b)
For the question, you need to:
1. Complete the preprocessing code below.
2. **In the PDF report**, briefly describe what preprocessing operations you used and what are the purposes of them.

Hint: 
1. Only two operations are necessary to complete the basic preprocessing here.
2. The raw input read from the dataset will be PIL images.
3. Data augmentation operations are not mendatory, but feel free to incorporate them if you want.
4. Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [6]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

transform_val = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean, std)]
)
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [7]:
# do NOT change these
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, mode="train", download=True, transform=transform_train
)
val_set = CIFAR10(root=DATA_ROOT, mode="val", download=True, transform=transform_val)

# construct dataloader
train_loader = DataLoader(
    train_set,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,  # your code
    num_workers=4,
)
val_loader = DataLoader(
    val_set,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
    num_workers=4,  # your code  # your code
)
#############################################

Using downloaded and verified file: ./data\cifar10_trainval_F22.zip
Extracting ./data\cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data\cifar10_trainval_F22.zip
Extracting ./data\cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

In [8]:
# specify the device for computation
#############################################
# your code here
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for computation")

model = SimpleNN().to(device)
#############################################

Using cuda:0 for computation


In [9]:
import torch

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Check the current device
print("Current device:", torch.cuda.current_device())

# Check the name of the current device
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Current device: 0
Device name: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [10]:
print(torch.__version__)
print(torch.version.cuda)

2.4.1+cu124
12.4


After mounting the model on GPU we can see that a new process on the GPU is created.

|    0   N/A  N/A     30548      C   D:\miniconda3\python 
exe                    N/A      |

|    0   N/A  N/A     35376    C+G   ...ience\NVIDIA GeForce Experience.exe      N/A      |

(hw2) 

## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum. 

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [11]:
import torch.nn as nn
import torch.optim as optim

# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.01
LR_LIST = [1.0, 0.1, 0.05, 0.01, 0.005, 0.001]
# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4
REG_LIST = [1e-2, 1e-3, 1e-4, 1e-5, 0.0]
#############################################
# your code here
# create loss function
criterion = nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(
    model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG
)
#############################################

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps: 

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


In [12]:
def train_val(
    optimizer,
    criterion,
    model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    INITIAL_LR=INITIAL_LR,
    REG=REG,
):
    # some hyperparameters
    # total number of training epochs

    EPOCHS = 30


    # the folder where the trained model is saved


    CHECKPOINT_FOLDER = "./saved_model"


    # start the training/validation process


    # the process should take about 5 minutes on a GTX 1070-Ti
    # if the code is written efficiently.
    best_val_acc = 0

    current_learning_rate = INITIAL_LR


    print("==> Training starts!")
    print("=" * 50)


    for i in range(0, EPOCHS):
        #######################


        # your code here
        # switch to train mode
        model.train()
        #######################


        print("Epoch %d:" % i)
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0


        train_loss = 0  # track training loss if you want

        # Train the model for 1 epoch.


        for batch_idx, (inputs, targets) in enumerate(train_loader):
            ####################################


            # your code here
            # copy inputs to device
            inputs, targets = inputs.to(device), targets.to(device)
            # compute the output and loss


            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # zero the gradient


            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()


            train_loss += loss.item()

            # count the number of correctly predicted samples in the current batch
            total_examples += targets.size(0)
            _, predicted = outputs.max(1)
            correct_examples += predicted.eq(targets).sum().item()
            ####################################


        print(
            f"CUDA memory allocated: {torch.cuda.memory_allocated(device) / 1e6:.2f} MB"
        )
        avg_loss = train_loss / len(train_loader)
        avg_acc = correct_examples / total_examples
        print("Training loss: %.4f, Training accuracy: %.4f" % (avg_loss, avg_acc))


        # Validate on the validation dataset
        #######################


        # your code here
        # switch to eval mode
        model.eval()

        #######################


        # this help you compute the validation accuracy
        total_examples = 0
        correct_examples = 0


        val_loss = 0  # again, track the validation loss if you want


        # disable gradient during validation, which can save GPU memory

        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                ####################################


                # your code here
                # copy inputs to device
                inputs, targets = inputs.to(device), targets.to(device)
                # compute the output and loss


                outputs = model(inputs)
                loss = criterion(outputs, targets)


                # count the number of correctly predicted samples in the current batch


                total_examples += targets.size(0)
                _, predicted = outputs.max(1)
                correct_examples += predicted.eq(targets).sum().item()
                val_loss += loss.item()
                ####################################


        avg_loss = val_loss / len(val_loader)
        avg_acc = correct_examples / total_examples

        print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))


        # save the model checkpoint


        if avg_acc > best_val_acc:
            best_val_acc = avg_acc

            if not os.path.exists(CHECKPOINT_FOLDER):
                os.makedirs(CHECKPOINT_FOLDER)
            print("Saving ...")

            state = {
                "state_dict": model.state_dict(),
                "epoch": i,
                "lr": current_learning_rate,
            }
            checkpoint_path = os.path.join(
                CHECKPOINT_FOLDER, f"LR{INITIAL_LR}_REG{REG}.pth"
            )
            torch.save(state, checkpoint_path)

        print("")


    print("=" * 50)


    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

In [13]:
for lr in LR_LIST:
    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(), lr=lr, momentum=MOMENTUM, weight_decay=REG
    )
    print(f"Training with learning rate: {lr}")
    train_val(optimizer, criterion, model, INITIAL_LR=lr)
for reg in REG_LIST:
    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=reg
    )
    print(f"Training with regularization strength: {reg}")
    train_val(optimizer, criterion, model, REG=reg)

Training with learning rate: 1.0
==> Training starts!
Epoch 0:
CUDA memory allocated: 18.93 MB
Training loss: 52.6282, Training accuracy: 0.1044
Validation loss: 2.3158, Validation accuracy: 0.0978
Saving ...

Epoch 1:
CUDA memory allocated: 18.93 MB
Training loss: 2.3208, Training accuracy: 0.1002
Validation loss: 2.3130, Validation accuracy: 0.1054
Saving ...

Epoch 2:
CUDA memory allocated: 18.93 MB
Training loss: 2.3214, Training accuracy: 0.1007
Validation loss: 2.3244, Validation accuracy: 0.0986

Epoch 3:
CUDA memory allocated: 18.93 MB
Training loss: 2.3199, Training accuracy: 0.1010
Validation loss: 2.3270, Validation accuracy: 0.1014

Epoch 4:
CUDA memory allocated: 18.93 MB
Training loss: 2.3186, Training accuracy: 0.0998
Validation loss: 2.3446, Validation accuracy: 0.1046

==> Optimization finished! Best validation accuracy: 0.1054
Training with learning rate: 0.1
==> Training starts!
Epoch 0:
CUDA memory allocated: 18.93 MB
Training loss: 1.6637, Training accuracy: 0.3838

KeyboardInterrupt: 

# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```